In [12]:
import numpy as np
import pandas as pd
import os

## Load & PreProcessing Data

In [13]:
def pre_process(df):
    # check OpenInt values
    if df[df.OpenInt > 0].empty: print("all OpenInt values are zero")
    # remove OpenInt column
    df.drop("OpenInt",axis=1, inplace=True)
    # remove High, low columns
    df.drop(["High",'Low'],axis=1, inplace=True)
    # add direction column : 1 if Close price >= Open price else 0
    df.loc[df.Close >= df.Open, 'direction'] = 1
    df.loc[df.Close < df.Open, 'direction'] = 0

# part 1

In [14]:
# choose stock to work with
# maximum = -1*float("inf")
# filename_max = ''
# dirname_max = ''
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for index, filename in enumerate(filenames):
#         try:
#             curr_df = pd.read_csv(os.path.join(dirname, filename))
#             curr_len = len(curr_df.index)
#             if curr_len > maximum:
#                 maximum, filename_max, dirname_max = curr_len, filename, dirname
#         except Exception as e:
#             print(e)

In [15]:
# print(maximum, filename_max, dirname)

In [16]:
df = pd.read_csv('ibm.us.txt', parse_dates=['Date'], index_col=['index'])
df

,Date,Open,High,Low,Close,Volume,OpenInt
index,,,,,,,
0,1962-01-02,6.4130,6.4130,6.3378,6.3378,467056,0
1,1962-01-03,6.3378,6.3963,6.3378,6.3963,350294,0
2,1962-01-04,6.3963,6.3963,6.3295,6.3295,314365,0
3,1962-01-05,6.3211,6.3211,6.1958,6.2041,440112,0
4,1962-01-08,6.2041,6.2041,6.0373,6.0870,655676,0
...,...,...,...,...,...,...,...
14054,2017-11-06,150.2700,150.3200,148.7900,149.3500,4543499,0
14055,2017-11-07,149.8700,150.0100,149.0100,149.8500,3729501,0
14056,2017-11-08,150.1000,150.2800,148.7900,150.0700,4679520,0


In [17]:
pre_process(df)
# remove High, low columns
df.drop(["Close"],axis=1, inplace=True)

all OpenInt values are zero


In [18]:
df

,Date,Open,Volume,direction
index,,,,
0,1962-01-02,6.4130,467056,0.0
1,1962-01-03,6.3378,350294,1.0
2,1962-01-04,6.3963,314365,0.0
3,1962-01-05,6.3211,440112,0.0
4,1962-01-08,6.2041,655676,0.0
...,...,...,...,...
14054,2017-11-06,150.2700,4543499,0.0
14055,2017-11-07,149.8700,3729501,0.0
14056,2017-11-08,150.1000,4679520,0.0


In [19]:
# check days range data
print(f'Data from day {min(df.Date)} to day {max(df.Date)}')

Data from day 1962-01-02 00:00:00 to day 2017-11-10 00:00:00


In [20]:
df['weekDay'] = df.Date.dt.weekday

In [21]:
week = set()
week_index = 0
for index,row in df.iterrows():
    if row.weekDay not in week and all([x < row.weekDay for x in week]):
        week.add(row.weekDay)
        df.at[index, 'weekNum'] = week_index
    else:
        week_index = week_index + 1
        week = set()
        week.add(row.weekDay)
        df.at[index, 'weekNum'] = week_index

In [22]:
for index in range(int(max(df.weekNum))+1):
    week_days = df[df.weekNum == index] 
    if len(week_days) != 5:
        df.drop(df[df.weekNum == index].index, inplace=True)

In [23]:
df.reset_index(inplace=True, drop=True)

In [24]:
week_days_index = 0
for index,row in df.iterrows():
    df.at[index, 'weekNum'] = week_days_index
    if (index+1) % 5 == 0:# and index != 0:
        week_days_index += 1

In [25]:
df.drop(["Date"],axis=1, inplace=True)

In [26]:
num_of_weeks = max(df.weekNum)

In [27]:
df['Open'] = df.groupby('weekNum')['Open'].apply(list)
df['Volume'] = df.groupby('weekNum')['Volume'].apply(list)
df['direction'] = df.groupby('weekNum')['direction'].apply(list)

In [28]:
df.drop(["weekDay", 'weekNum'],axis=1, inplace=True)
df.reset_index(inplace=True, drop=True)
df = df.head(int(num_of_weeks))

In [29]:
df

,Open,Volume,direction
0,"[6.2041, 6.1208, 6.1707, 6.1875, 6.2543]","[655676, 592806, 359274, 386220, 529933]","[0.0, 1.0, 1.0, 1.0, 1.0]"
1,"[6.2708, 6.2708, 6.1875, 6.1291, 6.1291]","[305383, 305383, 502984, 449093, 485021]","[1.0, 0.0, 0.0, 1.0, 1.0]"
2,"[6.1374, 6.1208, 6.0624, 6.0956, 6.0287]","[332329, 449093, 494001, 386220, 296401]","[0.0, 0.0, 1.0, 0.0, 0.0]"
3,"[5.9951, 5.8952, 5.8784, 6.0703, 6.1457]","[700585, 889207, 916151, 772443, 610767]","[0.0, 0.0, 1.0, 1.0, 1.0]"
4,"[6.1875, 6.104, 6.1208, 6.1208, 6.1208]","[377238, 314365, 305383, 26945, 520952]","[0.0, 1.0, 1.0, 1.0, 1.0]"
...,...,...,...
2403,"[143.91, 145.24, 145.55, 145.23, 145.19]","[3002790, 2325614, 2265636, 2713337, 2647099]","[1.0, 1.0, 0.0, 1.0, 0.0]"
2404,"[145.05, 146.25, 146.93, 146.1, 146.02]","[2709319, 4069894, 3738253, 3337684, 2530990]","[1.0, 1.0, 0.0, 0.0, 0.0]"
2405,"[145.76, 145.18, 155.57, 158.22, 159.48]","[3072486, 6266166, 30778036, 10009756, 7945358]","[0.0, 0.0, 1.0, 1.0, 1.0]"
2406,"[160.45, 158.12, 154.47, 152.78, 153.01]","[5835986, 8257417, 6987016, 4341600, 4944066]","[0.0, 0.0, 0.0, 0.0, 0.0]"


# part 2

In [30]:
df_ibm = pd.read_csv(os.path.join('/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks', 'ibm.us.txt'), parse_dates=['Date'])
df_crm = pd.read_csv(os.path.join('/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks', 'crm.us.txt'), parse_dates=['Date'])
df_sap = pd.read_csv(os.path.join('/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks', 'sap.us.txt'), parse_dates=['Date'])
df_orcl = pd.read_csv(os.path.join('/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks', 'orcl.us.txt'), parse_dates=['Date'])
df_msft = pd.read_csv(os.path.join('/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks', 'msft.us.txt'), parse_dates=['Date'])
df_acn = pd.read_csv(os.path.join('/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks', 'acn.us.txt'), parse_dates=['Date'])

FileNotFoundError: [Errno 2] File b'/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks\\ibm.us.txt' does not exist: b'/kaggle/input/price-volume-data-for-all-us-stocks-etfs/Stocks\\ibm.us.txt'

In [ ]:
stocks_df = [df_ibm, df_crm, df_sap, df_orcl, df_msft, df_acn]

In [ ]:
for df in stocks_df:
    pre_process(df)
    # compute Change Colum
    df['Change'] = df.Close - df.Open
    # remove High, low columns
    df.drop(["Close"],axis=1, inplace=True)

In [ ]:
for index, df in enumerate(stocks_df):
    if index == 0:
        minimum = min(df.Date)
        maximum = max(df.Date)
    if index != 0 and min(df.Date) > minimum:
        minimum = min(df.Date)
    if index != 0 and max(df.Date) < maximum:
        maximum = max(df.Date)

In [ ]:
df_ibm['Date'].intersection(df_crm['Date'])

In [ ]:
set(df_ibm)

In [ ]:
pd.Series(list(set(df_ibm) & set(df_crm)))